In [16]:
import pandas as pd
import numpy as np

In [17]:
#Catálogos de OrderEntry y Reparto

UO = 'MR0068'
CS1 = 'MJCQ'
CS2 = ''
CS3 = ''

cat_oe = pd.read_csv('Catalogo_OE.csv')
cat_rep = pd.read_csv('Catalogo_Reparto.csv')

#Base de datos Clientes SQL
planner = pd.read_csv('Planner_{}.csv'.format(UO))

#Bases de datos Greenmile
cs_1 = pd.read_csv('{}.csv'.format(CS1),encoding='latin-1')

try:
    cs_2 = pd.read_csv('{}.csv'.format(CS2), encoding='latin-1')
except FileNotFoundError:
    print("Archivo cs_2.csv no encontrado. Se omitirá.")
    cs_2 = pd.DataFrame()

try:
    cs_3 = pd.read_csv('{}.csv'.format(CS3), encoding='latin-1')
except FileNotFoundError:
    print("Archivo cs_3.csv no encontrado. Se omitirá.")
    cs_3 = pd.DataFrame()

greenmile = pd.concat([cs_1,cs_2,cs_3], ignore_index=True)

#Base de GEC RTM
df_gec = pd.read_csv('Catalogo_GEC_RTM.csv')

C:\Users\MX05081363\AppData\Local\Temp\ipykernel_31180\854068287.py:15: DtypeWarning: Columns (1,2,12) have mixed types. Specify dtype option on import or set low_memory=False.
  cs_1 = pd.read_csv('{}.csv'.format(CS1),encoding='latin-1')


Archivo cs_2.csv no encontrado. Se omitirá.
Archivo cs_3.csv no encontrado. Se omitirá.


C:\Users\MX05081363\AppData\Local\Temp\ipykernel_31180\854068287.py:32: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gec = pd.read_csv('Catalogo_GEC_RTM.csv')


In [18]:
# Seleccionar y renombrar columnas relevantes
greenmile = greenmile[['CÃ³digo de la UbicaciÃ³n', 'Longitud', 'Latitud', 'OrganizaciÃ³n']].copy()
greenmile.columns = ['Cliente GM', 'Longitud GM', 'Latitud GM', 'UO GM']

# Rellenar valores nulos en coordenadas con 0
greenmile['Longitud GM'] = greenmile['Longitud GM'].fillna(0)
greenmile['Latitud GM'] = greenmile['Latitud GM'].fillna(0)

# Convertir 'Cliente' a numérico, eliminando filas con errores
greenmile['Cliente GM'] = pd.to_numeric(greenmile['Cliente GM'], errors='coerce')
greenmile = greenmile.dropna(subset=['Cliente GM'])
greenmile['Cliente GM'] = greenmile['Cliente GM'].astype(int)

greenmile = greenmile.drop_duplicates(subset='Cliente GM', keep='first')
greenmile = greenmile.drop_duplicates(subset='Latitud GM')
greenmile = greenmile.drop_duplicates(subset='Longitud GM')

In [19]:
planner['DIRECCIÓN'] = planner['DIRECCIÓN'].fillna('DESCONOCIDO')
planner['ENTRE_CALLE_1'] = planner['ENTRE_CALLE_1'].fillna('DESCONOCIDO')
planner['ENTRE_CALLE_2'] = planner['ENTRE_CALLE_2'].fillna('DESCONOCIDO')
planner['DELEGACION_MUNICIPIO'] = planner['DELEGACION_MUNICIPIO'].fillna('DESCONOCIDO')
planner['ID_VP'] = planner['ID_VP'].fillna('SIN VP')
planner['MODALIDAD'] = planner['MODALIDAD'].fillna('NO DATA')
planner['TPV'] = planner['TPV'].fillna('NO DATA')
planner['RUTA'] = planner['RUTA'].fillna('NO DATA')
planner['FV'] = planner['FV'].fillna('NO DATA')
planner['RITMO'] = planner['RITMO'].fillna('NO DATA')
planner['MÉTODO_VENTA'] = planner['MÉTODO_VENTA'].fillna('NO DATA')
planner['ID_MULTICENTRO'] = planner['ID_MULTICENTRO'].fillna('NO ASIGNADO')

df_gec_renamed = df_gec.rename(columns={'CLIENTE': 'ID_SAP'})
planner = pd.merge(planner, right=df_gec_renamed, how='left', on='ID_SAP')

# Rellenamos los valores faltantes del GEC nuevo (por ejemplo, GEC_RTM) con el GEC anterior
planner['GEC_RTM'] = planner['GEC_RTM'].fillna(planner['GEC_SAP'])



In [20]:
columnas_basicas = [
    'ID_SAP', 'CLIENTE', 'FECHA_ALTA', 'DIRECCIÓN', 'ENTRE_CALLE_1', 'ENTRE_CALLE_2','COLONIA',
    'DELEGACION_MUNICIPIO', 'CODIGO_POSTAL', 'ESTATUS', 'ID_VP', 'MODALIDAD',
    'GEC_SAP','GEC_RTM','_MADUREZ_CTE','Estatus_Cte', 'CADENA', 'DESC_CADENA','CANAL_KOF','CANAL_SAP', 'GRUPO_RM1', 'GRUPO_RM2',
    'ZONA', 'REGIÓN', 'REGIÓN_KOF_CVE', 'UO', 'CENTRO','GERENCIA','JEFATURA',
    'CENTRO_SUMINISTRO','CONDICIÓN_EXPEDICIÓN','TIPO_SERVICIO','ID_MULTICENTRO', 'COORD_X', 'COORD_Y'
]
columnas_tpv = ['ID_SAP', 'TPV', 'RUTA', 'FV', 'RITMO', 'MÉTODO_VENTA']
df_tpv = planner[columnas_tpv]
df_basico = planner[columnas_basicas].drop_duplicates(subset='ID_SAP')

df_basico = df_basico.drop_duplicates(subset='ID_SAP')

# Pivot con múltiples valores
df_pivot = df_tpv.pivot_table(index='ID_SAP',
                          columns='TPV',
                          values=['RUTA', 'FV', 'RITMO', 'MÉTODO_VENTA'],
                          aggfunc='first')

# Aplanar columnas
df_pivot.columns = [f'{col[0].capitalize()} {col[1]}' for col in df_pivot.columns]

 # Reordenar columnas por TPV preferido
tpv_preferido = ['ZPV', 'ZTK', 'ZJV','ZTV','ZGH','ZGC','ZGF','ZPG','ZTJ','ZED','ZIV','ZDD','ZGC','ZHB','ZTF','ZVE','ZVS','ZDY']
columnas_validas = [col for col in df_pivot.columns if ' ' in col]
tpvs_encontrados = {col.split(' ')[1] for col in columnas_validas}
otros_tpv = sorted(tpvs_encontrados - set(tpv_preferido))
nuevo_orden = tpv_preferido + otros_tpv

 # Variables esperadas
variables = ['Ruta', 'Fv', 'Ritmo', 'Método_venta']
orden_columnas = []
for tpv in nuevo_orden:
    for var in variables:
        col_name = f'{var} {tpv}'
        if col_name in df_pivot.columns:
            orden_columnas.append(col_name)

df_pivot = df_pivot[orden_columnas]
df_pivot = df_pivot.reset_index()

df_pivot = df_pivot.drop_duplicates(subset='ID_SAP')

# Reemplazar NaN con guion medio
df_pivot = df_pivot.fillna('-')

df_pivot = df_pivot.drop_duplicates(subset='ID_SAP')
df_basico = df_basico.drop_duplicates(subset='ID_SAP')

# Unir datos básicos con pivot
df_final = pd.merge(df_basico, df_pivot, on='ID_SAP', how='left')

# Eliminar columnas duplicadas (conservando la primera)
duplicados = df_final.columns[df_final.columns.duplicated()]
if len(duplicados) > 0:
    print("❌ Columnas duplicadas encontradas:", duplicados.tolist())
    df_final = df_final.loc[:, ~df_final.columns.duplicated()]
    print("✅ Columnas duplicadas eliminadas.")
else:
    print("✅ No se encontraron columnas duplicadas.")

# Días de la semana
dias_semana = ['L', 'M', 'R', 'J', 'V', 'S', 'D']

# Columnas FV por TPV
columnas_fv = [col for col in df_final.columns if col.startswith('Fv ')]

# Insertar columnas nuevas justo después de cada columna Fv
for col_fv in columnas_fv:
    tpv = col_fv.split(' ')[1]  # Ej. 'ZGC'
    
    # Generar nuevas columnas
    nuevas_columnas = {}
    for dia in dias_semana:
        nueva_col = f'{tpv}_{dia}'
        nuevas_columnas[nueva_col] = df_final[col_fv].apply(lambda x: 1 if dia in str(x) else 0)
    
    # Encontrar la posición donde está la columna 'Fv <TPV>'
    idx = df_final.columns.get_loc(col_fv) + 1
    
    # Insertar las columnas en esa posición
    for i, (col_name, serie) in enumerate(nuevas_columnas.items()):
        df_final.insert(loc=idx + i, column=col_name, value=serie)

# Mostrar para validar
print(df_final.head())


❌ Columnas duplicadas encontradas: ['Ruta ZGC', 'Fv ZGC', 'Ritmo ZGC', 'Método_venta ZGC']
✅ Columnas duplicadas eliminadas.
      ID_SAP                         CLIENTE  FECHA_ALTA    DIRECCIÓN  \
0  660603349              Ramon Zavala Loeza  05.11.2011  DESCONOCIDO   
1  660603366      Luis Alberto Vargas Zurita  05.11.2011  DESCONOCIDO   
2  670001322  MANUEL ALEJANDRO GARCIA GODINE  05.11.2011  DESCONOCIDO   
3  670001845           Miguel Alcaraz Chavez  05.11.2011  DESCONOCIDO   
4  670001848          Roberto Mejía Cisneros  05.11.2011  DESCONOCIDO   

  ENTRE_CALLE_1 ENTRE_CALLE_2 COLONIA DELEGACION_MUNICIPIO  CODIGO_POSTAL  \
0   DESCONOCIDO   DESCONOCIDO     NaN          DESCONOCIDO          58200   
1   DESCONOCIDO   DESCONOCIDO     NaN          DESCONOCIDO          58200   
2   DESCONOCIDO   DESCONOCIDO     NaN          DESCONOCIDO          58200   
3   DESCONOCIDO   DESCONOCIDO     NaN          DESCONOCIDO          58200   
4   DESCONOCIDO   DESCONOCIDO     NaN          DESC

C:\Users\MX05081363\AppData\Local\Temp\ipykernel_31180\1562154128.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final.insert(loc=idx + i, column=col_name, value=serie)
C:\Users\MX05081363\AppData\Local\Temp\ipykernel_31180\1562154128.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final.insert(loc=idx + i, column=col_name, value=serie)
C:\Users\MX05081363\AppData\Local\Temp\ipykernel_31180\1562154128.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

In [21]:
df_coordenadas = pd.merge(left = df_final, right = greenmile, how = 'left', left_on = 'ID_SAP', right_on = 'Cliente GM')

df_coordenadas.drop(columns = ['Cliente GM','UO GM'], inplace = True)

duplicados_gm = greenmile[greenmile.duplicated(subset='Cliente GM', keep=False)]
print(duplicados_gm.sort_values('Cliente GM'))
print(f'Total de duplicados: {duplicados_gm["Cliente GM"].nunique()}')

assert df_coordenadas['ID_SAP'].is_unique, "¡Aún hay duplicados en ID_SAP!"

Empty DataFrame
Columns: [Cliente GM, Longitud GM, Latitud GM, UO GM]
Index: []
Total de duplicados: 0


In [22]:
# Crear la nueva columna "Latitud Final" en el DataFrame df_coordenadas
df_coordenadas['Latitud Final'] = np.where(
    (df_coordenadas['Latitud GM'] == 0) | (df_coordenadas['Latitud GM'].isna()),
    df_coordenadas['COORD_Y'],
    df_coordenadas['Latitud GM']
)

# Crear la nueva columna "Latitud Final" en el DataFrame df_coordenadas
df_coordenadas['Longitud Final'] = np.where(
    (df_coordenadas['Longitud GM'] == 0) | (df_coordenadas['Longitud GM'].isna()),
    df_coordenadas['COORD_X'],
    df_coordenadas['Longitud GM']
)

df_coordenadas.drop(columns = ['Latitud GM','Longitud GM','COORD_Y','COORD_X'], inplace = True)

In [23]:
cat_oe = cat_oe[['Ruta','Tipo','Descripción Tipo']].copy()
cat_rep = cat_rep[['RutaReparto','EsquemaReparto']].copy()

In [24]:
def merge_por_order_entry(left_df, right_df, claves_left, clave_right='Ruta'):
    resultado = left_df.copy()
    merged_cols = [col for col in right_df.columns if col != clave_right]
    resultado[merged_cols] = None  # Inicializa columnas vacías

    sin_match = pd.Series([True] * len(resultado))

    for clave in claves_left:
        temp_merge = left_df[sin_match].merge(right_df, left_on=clave, right_on=clave_right, how='left')
        for col in merged_cols:
            resultado.loc[sin_match, col] = temp_merge[col].values
        sin_match = resultado[merged_cols[0]].isna()  # puedes mejorar esto si quieres validar múltiples columnas

    return resultado

In [25]:
df_match_oe = merge_por_order_entry(df_coordenadas, cat_oe, ['Ruta ZPV', 'Ruta ZIV', 'Ruta ZED'])

In [26]:
def merge_por_reparto(left_df, right_df, claves_left, clave_right='RutaReparto'):
    resultado = left_df.copy()
    merged_cols = [col for col in right_df.columns if col != clave_right]
    resultado[merged_cols] = None  # Inicializa columnas vacías

    sin_match = pd.Series([True] * len(resultado))

    for clave in claves_left:
        temp_merge = left_df[sin_match].merge(right_df, left_on=clave, right_on=clave_right, how='left')
        for col in merged_cols:
            resultado.loc[sin_match, col] = temp_merge[col].values
        sin_match = resultado[merged_cols[0]].isna()  # puedes mejorar esto si quieres validar múltiples columnas

    return resultado

In [27]:
df_match_reparto = merge_por_reparto(df_match_oe, cat_rep, ['Ruta ZDD', 'Ruta ZVE'])

In [28]:
df_biblia_full = df_match_reparto
df_biblia_full.to_csv('Planner_Biblia_{}.csv'.format(UO), index=False, encoding='utf-8-sig')

In [29]:
df_biblia_tradicional = df_biblia_full[df_biblia_full['Tipo'].isin(['PR', 'PCOP', 'PROP'])]
df_biblia_tradicional.to_csv('Planner_Biblia_Tradicional_{}.csv'.format(UO), index=False, encoding='utf-8-sig')

In [30]:
df_vp_report = planner.pivot_table(index=['ID_VP','RUTA'], values='ID_SAP', aggfunc='count')
df_vp_report = df_vp_report.reset_index()
df_vp_report['ID_VP'] = df_vp_report['ID_VP'].astype(str)

print(df_vp_report[df_vp_report['ID_SAP']>500])

          ID_VP    RUTA  ID_SAP
209  60023555.0  QUWA05     712
212  60023558.0  QUWA11     623
213  60023562.0  QUWA14     785
215  60023564.0  QUWA22     783
216  60023565.0  QUWA24     652
217  60023566.0  QUWA35     539
223  60027573.0  QUWA03     741
226  60027576.0  QUWA02     781
227  60027577.0  QUWA01     760
228  60027578.0  QUWA13     642
229  60027579.0  QUWA04     777
